### Nesse miniprojeto será implementado um algoritmo para filtragem colaborativa afim de construir um sistema de recomendação

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from numpy import loadtxt

|Notação <br />  geral  | Descrição| Python (se houver) |
|:-------------|:------------------------------------------------------------||
| $r(i,j)$     | scalar; = 1  se user j classificar filme, i  = 0  se não             ||
| $y(i,j)$     | scalar; = classificação dada pelo user j para filme  i    (se r(i,j) = 1 for definido) ||
|$\mathbf{w}^{(j)}$ | vector; parametros user j ||
|$b^{(j)}$     |  scalar; parametro para user j ||
| $\mathbf{x}^{(i)}$ |   vector; nota atribuida para o filme i        ||     
| $n_u$        | número de usuário |num_users|
| $n_m$        | número de filmes | num_movies |
| $n$          | número de variáveis | num_features                    |
| $\mathbf{X}$ |  matriz de vetores $\mathbf{x}^{(i)}$         | X |
| $\mathbf{W}$ |  matriz de vetores $\mathbf{w}^{(j)}$         | W |
| $\mathbf{b}$ |  vetor de erro para o parâmetro $b^{(j)}$ | b |
| $\mathbf{R}$ | matriz de elementos $r(i,j)$                    | R |


O algoritmo de aprendizado de filtragem colaborativa é aplicado a um conjunto de dados de classificações de filmes. O objetivo do sistema de recomendação por filtragem colaborativa é gerar dois vetores: Para cada usuário, um 'vetor de parâmetros' que incorpora os gostos de filme de um usuário. Para cada filme, um vetor de características do mesmo tamanho que incorpora alguma descrição do filme. O produto escalar dos dois vetores mais o termo de viés deve produzir uma estimativa da classificação que o usuário pode atribuir a esse filme. O diagrama abaixo detalha como esses vetores são aprendidos

<figure>
   <img src="./images/ColabFilterLearn1.PNG"  style="width:740px;height:250px;" >
</figure>

As classificações existentes são fornecidas em forma de matriz, conforme mostrado. $𝑌$ contém as classificações; 0,5 a 5, em passos de 0,5. Classificação 0 se o filme não foi classificado. $𝑅$ contém o valor 1 onde os filmes foram classificados. Os filmes estão em linhas, os usuários em colunas. Cada usuário tem um vetor de parâmetro $w^{user}$ e bias. Cada filme tem um vetor de recursos $x^{movie}$. Esses vetores são aprendidos simultaneamente usando as classificações de usuário/filme existentes, conforme dados de treinamento. Um exemplo de treinamento é mostrado acima: $\mathbf{w}^{(1)} \cdot \mathbf{x}^{(1)} + b^{(1)} = 4$. O vetor de recursos $x^{movie}$ deve satisfazer todos os usuários, enquanto o vetor de usuário $w^{user}$ deve satisfazer todos os filmes. Esta é a origem do nome desta abordagem - todos os usuários colaboram para gerar o conjunto de classificação.

Uma vez que os vetores de recursos e parâmetros são aprendidos, eles podem ser usados para prever como um usuário pode classificar um filme sem classificação.

### Conjunto de dados

O conjunto de dados é derivado do conjunto de dados MovieLens [MovieLens "ml-latest-small"](https://grouplens.org/datasets/movielens/latest/).   
[F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>]

O conjunto de dados original tem 9.000 filmes avaliados por 600 usuários. O tamanho do conjunto de dados foi reduzido para se concentrar em filmes dos anos desde 2000. Esse conjunto de dados consiste em classificações em uma escala de 0,5 a 5 em incrementos de 0,5 passos. O conjunto de dados reduzido tem 𝑛$n_u = 443$ usuários e $n_m= 4778$ filmes.

A matriz $𝑌$ (uma matriz $n_m \times n_u$) armazena as classificações $y^{(i,j)}$ . A matriz $𝑅$ é uma matriz indicadora de valor binário, onde $R(i,j) = 1$ se o usuário $𝑗$ deu uma classificação ao filme $𝑖$ , e $R(i,j)=0$ caso contrário.

Também é utilizado as matrizes $\mathbf{X}$, $\mathbf{W}$ and $\mathbf{b}$: 

$$\mathbf{X} = 
\begin{bmatrix}
--- (\mathbf{x}^{(0)})^T --- \\
--- (\mathbf{x}^{(1)})^T --- \\
\vdots \\
--- (\mathbf{x}^{(n_m-1)})^T --- \\
\end{bmatrix} , \quad
\mathbf{W} = 
\begin{bmatrix}
--- (\mathbf{w}^{(0)})^T --- \\
--- (\mathbf{w}^{(1)})^T --- \\
\vdots \\
--- (\mathbf{w}^{(n_u-1)})^T --- \\
\end{bmatrix},\quad
\mathbf{ b} = 
\begin{bmatrix}
 b^{(0)}  \\
 b^{(1)} \\
\vdots \\
b^{(n_u-1)} \\
\end{bmatrix}\quad
$$

A $i$-ésima linha de $\mathbf{X}$ corresponde ao vetor de recursos $x^{(i)}$ para o $i$-ésimo filme e $j$-ésima linha de $\mathbf{W}$ correspondente ao vetor $\mathbf{w}^{(j)}$, para o $j$ usuário. $x^{(i)}$ e $\mathbf{w}^{(j)}$ são vetores $n$-dimensional. Nesse projeto será usado $n=10$, portanto $\mathbf{x}^{(i)}$ and $\mathbf{w}^{(j)}$ têm 10 elementos,  dessa forma, $\mathbf{X}$ é uma matriz $n_m \times 10$ e $\mathbf{W}$ é uma matriz $n_u \times 10$.

### Carregando base de dados

$Y$ e $R$ será carregado do conjunto de dados de classificações de filmes. $\mathbf{X}$, $\mathbf{W}$, e $\mathbf{b}$ será carregado com valores pré-calculados

In [2]:
def load__params():
    
    #Carrega os valores da matriz X
    file = open('./data/small_movies_X.csv', 'rb')
    X = loadtxt(file, delimiter = ",")
    
    #Carrega os valores da matriz W
    file = open('./data/small_movies_W.csv', 'rb')
    W = loadtxt(file,delimiter = ",")

    #Carrega os valores da matriz b
    file = open('./data/small_movies_b.csv', 'rb')
    b = loadtxt(file,delimiter = ",")
    b = b.reshape(1,-1)
    
    #Coleta número de filmes, variáveis e usuários
    num_movies, num_features = X.shape
    num_users,_ = W.shape
    
    return(X, W, b, num_movies, num_features, num_users)

In [3]:
def load_ratings():
    
    #Carrega os valores da matriz Y
    file = open('./data/small_movies_Y.csv', 'rb')
    Y = loadtxt(file,delimiter = ",")

    #Carrega os valores da matriz R
    file = open('./data/small_movies_R.csv', 'rb')
    R = loadtxt(file,delimiter = ",")
    
    return(Y,R)

In [4]:
#Carregando dados
X, W, b, num_movies, num_features, num_users = load__params()
Y, R = load_ratings()

print("Y", Y.shape)
print("R", R.shape)
print("X", X.shape)
print("W", W.shape)
print("b", b.shape)
print("Número de variáveis:", num_features)
print("Número de filmes:",   num_movies)
print("Número de usuários:",    num_users)

Y (4778, 443)
R (4778, 443)
X (4778, 10)
W (443, 10)
b (1, 443)
Número de variáveis: 10
Número de filmes: 4778
Número de usuários: 443


### Função de custo

A função de custo de filtragem colaborativa é dada por:

$$J({\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},\mathbf{w}^{(0)},b^{(0)},...,\mathbf{w}^{(n_u-1)},b^{(n_u-1)}})= \frac{1}{2}\sum_{(i,j):r(i,j)=1}(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2
+\underbrace{
\frac{\lambda}{2}
\sum_{j=0}^{n_u-1}\sum_{k=0}^{n-1}(\mathbf{w}^{(j)}_k)^2
+ \frac{\lambda}{2}\sum_{i=0}^{n_m-1}\sum_{k=0}^{n-1}(\mathbf{x}_k^{(i)})^2
}_{regularizacao}
\tag{1}$$
"Para todo $i$, $j$ onde $r(i,j)$ igual $1$" a função de custo pode ser escrita:

$$
= \frac{1}{2}\sum_{j=0}^{n_u-1} \sum_{i=0}^{n_m-1}r(i,j)*(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2
+\text{regularizacao}
$$

In [5]:
#Algoritmo da função de custo sem regularização
def cofi_cost_func(X, W, b, Y, R, lambda_):
    
    nm, nu = Y.shape
    J = 0
    
    for j in range(nu):
        w = W[j,:]
        b_j = b[0,j]
        for i in range(nm):
            x = X[i,:]
            y = Y[i,j]
            r = R[i,j]
            J += np.square(r * (np.dot(w,x) + b_j - y ) )
    J = J/2
    
    return J

In [6]:
# Avaliação da função de custo
num_users_r = 4
num_movies_r = 5 
num_features_r = 3

X_r = X[:num_movies_r, :num_features_r]
W_r = W[:num_users_r,  :num_features_r]
b_r = b[0, :num_users_r].reshape(1,-1)
Y_r = Y[:num_movies_r, :num_users_r]
R_r = R[:num_movies_r, :num_users_r]

J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Custo: {J:0.2f}")

Custo: 13.67


In [7]:
#Algoritmo da função de custo com regularização
def cofi_cost_func_reg(X, W, b, Y, R, lambda_):
    
    nm, nu = Y.shape
    J = 0
    
    for j in range(nu):
        w = W[j,:]
        b_j = b[0,j]
        for i in range(nm):
            x = X[i,:]
            y = Y[i,j]
            r = R[i,j]
            J += np.square(r * (np.dot(w,x) + b_j - y ) )
    
    J = J/2
    
    J += (lambda_/2) * (np.sum(np.square(W)) + np.sum(np.square(X)))
    
    return J

In [8]:
# Avaliação da função de custo com regularização
J = cofi_cost_func_reg(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Custo: {J:0.2f}")

Custo: 28.09


In [9]:
# Função de custo vetorizada

def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [10]:
# Avaliação da função de custo

# Avaliação da função de custo sem regularização
J = cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Custo: {J:0.2f}")

# Avaliação da função de custo com regularização
J = cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Custo: {J:0.2f}")

Custo: 13.67
Custo: 28.09


### Treinamento

In [11]:
#Função para carregar a lista de filmes
def load_Movie_List_pd():
    df = pd.read_csv('./data/small_movie_list.csv', header=0, index_col=0,  delimiter=',', quotechar='"')
    mlist = df["title"].to_list()
    return(mlist, df)

In [12]:
# Carrega lista de filmes
movieList, movieList_df = load_Movie_List_pd()

In [13]:
#Lista de filmes

movieList

['Yards, The (2000)',
 'Next Friday (2000)',
 'Supernova (2000)',
 'Down to You (2000)',
 'Scream 3 (2000)',
 'Boondock Saints, The (2000)',
 'Gun Shy (2000)',
 'Beach, The (2000)',
 'Snow Day (2000)',
 'Tigger Movie, The (2000)',
 'Boiler Room (2000)',
 'Hanging Up (2000)',
 'Pitch Black (2000)',
 'Whole Nine Yards, The (2000)',
 'Black Tar Heroin: The Dark End of the Street (2000)',
 'Reindeer Games (2000)',
 'Wonder Boys (2000)',
 'Chain of Fools (2000)',
 'Drowning Mona (2000)',
 'Next Best Thing, The (2000)',
 'What Planet Are You From? (2000)',
 'Closer You Get, The (2000)',
 'Mission to Mars (2000)',
 'Erin Brockovich (2000)',
 'Final Destination (2000)',
 'Romeo Must Die (2000)',
 'Here on Earth (2000)',
 'Whatever It Takes (2000)',
 'High Fidelity (2000)',
 'Road to El Dorado, The (2000)',
 'Skulls, The (2000)',
 'Frequency (2000)',
 'Ready to Rumble (2000)',
 'Return to Me (2000)',
 'Rules of Engagement (2000)',
 "Joe Gould's Secret (2000)",
 'Me Myself I (2000)',
 '28 Days (

In [14]:
#Dataframe de avaliações

movieList_df.head()

mean rating  number of ratings               title
0     3.400000                  5   Yards, The (2000)
1     3.250000                  6  Next Friday (2000)
2     2.000000                  4    Supernova (2000)
3     2.000000                  4  Down to You (2000)
4     2.672414                 29     Scream 3 (2000)

#### A partir das avaliações o algoritmo fará recomendações de filmes. A célula abaixo pode ser alterada, inserindo e classificando os filmes, na sequência o algoritmo fará a recomendação

In [15]:
#Inicializa as avaliações
my_ratings = np.zeros(num_movies)        


# No arquivo small_movie_list.csv contém a relação do id para cada filme do dataset
# Por exemplo, Toy Story 3 tem ID 2700 e avalie com nota 5, logo:
my_ratings[2700] = 5 # Toy Story 3

my_ratings[2609] = 2 # Persuasion

my_ratings[929]  = 5   # Lord of the Rings: The Return of the King

my_ratings[246]  = 5   # Shrek

my_ratings[2716] = 3   # Inception

my_ratings[1150] = 5   #  The Incredibles

my_ratings[382]  = 2   # Amelie

my_ratings[366]  = 5   # Harry Potter and the Sorcerer's Stone

my_ratings[622]  = 5   # Harry Potter and the Chamber of Secrets

my_ratings[988]  = 3   # Eternal Sunshine of the Spotless Mind 

my_ratings[2925] = 1   # Louis Theroux: Law & Disorder

my_ratings[2937] = 1   # Nothing to Declare

my_ratings[793]  = 5   # Pirates of the Caribbean: The Curse of the Black Pearl

my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]

print('\n Novas Classificações:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0 :
        print(f'Classificação {my_ratings[i]} para {movieList_df.loc[i,"title"]}');


 Novas Classificações:

Classificação 5.0 para Shrek (2001)
Classificação 5.0 para Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Classificação 2.0 para Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Classificação 5.0 para Harry Potter and the Chamber of Secrets (2002)
Classificação 5.0 para Pirates of the Caribbean: The Curse of the Black Pearl (2003)
Classificação 5.0 para Lord of the Rings: The Return of the King, The (2003)
Classificação 3.0 para Eternal Sunshine of the Spotless Mind (2004)
Classificação 5.0 para Incredibles, The (2004)
Classificação 2.0 para Persuasion (2007)
Classificação 5.0 para Toy Story 3 (2010)
Classificação 3.0 para Inception (2010)
Classificação 1.0 para Louis Theroux: Law & Disorder (2008)
Classificação 1.0 para Nothing to Declare (Rien à déclarer) (2010)


In [16]:
# Função para normalizar as classificações
def normalizacao(Y, R):

    Ymean = (np.sum(Y*R,axis=1)/(np.sum(R, axis=1)+1e-12)).reshape(-1,1)
    Ynorm = Y - np.multiply(Ymean, R) 
    return(Ynorm, Ymean)

#### Adicionando as avaliações nas matrizes $Y$ e $R$

In [17]:
# Carrega classificações
Y, R = load_ratings()

# Adiciona novas classificações para Y
Y = np.c_[my_ratings, Y]

# Adiciona novo usuário na matriz R
R = np.c_[(my_ratings != 0).astype(int), R]

# Normaliza conjunto de dados
Ynorm, Ymean = normalizacao(Y, R)

In [18]:
#  Seleciona parâmentros para treinamento
num_movies, num_users = Y.shape
num_features = 100
tf.random.set_seed(10) 

W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

# Inicializa otimização
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

#### Treinamento do modelo, aprendizado dos parametros $\mathbf{X}$, $\mathbf{W}$, e $\mathbf{b}$

In [20]:
#Total de iterações
iterations = 400

#Valor de lambda
lambda_ = 1

for iter in range(iterations):
    
    with tf.GradientTape() as tape:

        cost_value = cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)

    grads = tape.gradient( cost_value, [X,W,b] )

    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    if iter % 20 == 0:
        print(f"Training loss {iter}: {cost_value:0.1f}")

Training loss 0: 2507.1
Training loss 20: 2302.7
Training loss 40: 2167.1
Training loss 60: 2074.5
Training loss 80: 2009.4
Training loss 100: 1962.3
Training loss 120: 1927.3
Training loss 140: 1900.6
Training loss 160: 1879.9
Training loss 180: 1863.4
Training loss 200: 1850.0
Training loss 220: 1839.1
Training loss 240: 1829.9
Training loss 260: 1822.2
Training loss 280: 1815.6
Training loss 300: 1810.0
Training loss 320: 1805.1
Training loss 340: 1800.9
Training loss 360: 1797.1
Training loss 380: 1793.9


### Recomendações

In [21]:
# Fazendo previsões utilizandos os pesos obtidos durante treinamento
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

# Restaura a média
pm = p + Ymean

my_predictions = pm[:,0]

# Ordena as previsões
ix = tf.argsort(my_predictions, direction='DESCENDING')

for i in range(17):
    j = ix[i]
    if j not in my_rated:
        print(f'Classificação {my_predictions[j]:0.2f} para o filme {movieList[j]}')

print('\n\nOriginal vs Previsões:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print(f'Original {my_ratings[i]}, Previsão {my_predictions[i]:0.2f} for {movieList[i]}')

Classificação 4.73 para o filme Colourful (Karafuru) (2010)
Classificação 4.56 para o filme Delirium (2014)
Classificação 4.56 para o filme Laggies (2014)
Classificação 4.56 para o filme One I Love, The (2014)
Classificação 4.55 para o filme Into the Abyss (2011)
Classificação 4.55 para o filme Eichmann (2007)
Classificação 4.55 para o filme Battle Royale 2: Requiem (Batoru rowaiaru II: Chinkonka) (2003)
Classificação 4.53 para o filme Particle Fever (2013)
Classificação 4.52 para o filme Kung Fu Panda: Secrets of the Masters (2011)
Classificação 4.52 para o filme 'Salem's Lot (2004)


Original vs Previsões:

Original 5.0, Previsão 4.90 for Shrek (2001)
Original 5.0, Previsão 4.81 for Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Original 2.0, Previsão 2.20 for Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Original 5.0, Previsão 4.88 for Harry Potter and the Chamber of Secrets (2002)
Original 5.0, Previsão 4.88 for Pirates of the 

##### Na prática, informações adicionais podem ser utilizadas para aprimorar as previsões. Acima, as classificações previstas para as primeiras centenas de filmes estão em uma pequena faixa. Podemos aumentar, selecionando os melhores filmes, filmes com classificações médias altas e filmes com mais de 20 classificações.

In [23]:
filter=(movieList_df["number of ratings"] > 20)
movieList_df["pred"] = my_predictions
movieList_df = movieList_df.reindex(columns=["pred", "mean rating", "number of ratings", "title"])
movieList_df.loc[ix[:300]].loc[filter].sort_values("mean rating", ascending=False)

pred  mean rating  number of ratings  \
2112  4.176614     4.238255                149   
155   3.997637     4.155914                 93   
929   4.866685     4.118919                185   
2700  4.803468     4.109091                 55   
393   4.311531     4.106061                198   
653   4.441409     4.021277                188   
2804  4.074952     3.989362                 47   
773   4.255208     3.960993                141   
2649  4.041575     3.943396                 53   
1051  4.149456     3.913978                 93   
361   4.176650     3.871212                132   
246   4.895007     3.867647                170   
1150  4.851509     3.836000                125   
1521  4.085108     3.816901                 71   
1081  4.166738     3.803797                 79   
793   4.876715     3.778523                149   
366   4.810354     3.761682                107   
622   4.880169     3.598039                102   
1045  4.043242     3.576087                 92   

                                                  title  
2112                            Dark Knight, The (2008)  
155                                       Snatch (2000)  
929   Lord of the Rings: The Return of the King, The...  
2700                                 Toy Story 3 (2010)  
393   Lord of the Rings: The Fellowship of the Ring,...  
653       Lord of the Rings: The Two Towers, The (2002)  
2804  Harry Potter and the Deathly Hallows: Part 1 (...  
773                                 Finding Nemo (2003)  
2649                    How to Train Your Dragon (2010)  
1051    Harry Potter and the Prisoner of Azkaban (2004)  
361                               Monsters, Inc. (2001)  
246                                        Shrek (2001)  
1150                            Incredibles, The (2004)  
1521         Harry Potter and the Goblet of Fire (2005)  
1081                                Spider-Man 2 (2004)  
793   Pirates of the Caribbean: The Curse of the Bla...  
366   Harry Potter and the Sorcerer's Stone (a.k.a. ...  
622      Harry Potter and the Chamber of Secrets (2002)  
1045                                     Shrek 2 (2004)